In [1]:
# IMPORTS
import os
import math
import json
import argparse 
from glob import glob

import torch

import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import tqdm
from pathlib import PureWindowsPath, PurePosixPath

In [2]:
#!pip install -U openmim
#!mim install mmcv-full==1.7.0
!pip install -U openmim
!pip install /kaggle/input/mmcvfullwhl/mmcv_full-1.7.0-cp37-cp37m-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.9 MB/s eta 0:00:00
Processing /kaggle/input/mmcvfullwhl/mmcv_full-1.7.0-cp37-cp37m-linux_x86_64.whl


In [3]:
!pip install mmdet
!pip install mmpose
from mmpose.apis import inference_top_down_pose_model, init_pose_model
from mmdet.apis import inference_detector, init_detector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp37-cp37m-linux_x86_64.whl size=373766 sha256=9ed81cc519c94a925d496a28bf82f97e43e7a25b1f83dfe84280d63cbced78b0
  Stored in directory: /root/.cache/pip/wheels/06/f6/f9/9cc49c6de8e3cf27dfddd91bf46595a057141d4583a2adaf03
Successfully built pycocotools
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58285 sha256=9f26d521dfdce8eb68e9d5cac563040588e3d444377fe9d401d163e879945270
  Stored in directory: /root

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [4]:
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

package_paths = [
   '/kaggle/input/HRFormer',
]

for pth in package_paths:
    sys.path.append(pth)
    
!cp -r /kaggle/input/HRFormer/ /kaggle/working/hrformer/

cp: cannot stat '/kaggle/input/HRFormer/': No such file or directory


In [5]:
#!rm -r /kaggle/working/

In [6]:
"""
obj_test = pd.read_pickle('/kaggle/input/goorealdataset/valrealhumansNew.pickle', compression='infer')
df_rg_test = pd.DataFrame.from_records(obj_test)
print(len(df_rg_test))
df = df_rg_test
"""

obj_test = pd.read_pickle('/kaggle/input/goorealdataset/testrealhumansSparsedNew.pickle', compression='infer')
df_rg_test = pd.DataFrame.from_records(obj_test)
print(len(df_rg_test))
df = df_rg_test

"""
obj_test = pd.read_pickle('/kaggle/input/goosynthtestdataset/goosynth_test_v2_no_segm.pkl', compression='infer')
df_rg_test = pd.DataFrame.from_records(obj_test)
print(len(df_rg_test))
df = df_rg_test
"""

1162


"\nobj_test = pd.read_pickle('/kaggle/input/goosynthtestdataset/goosynth_test_v2_no_segm.pkl', compression='infer')\ndf_rg_test = pd.DataFrame.from_records(obj_test)\nprint(len(df_rg_test))\ndf = df_rg_test\n"

In [7]:
df.head()

,filename,width,height,ann,gaze_item,gazeIdx,gaze_cx,gaze_cy,hx,hy,seg,cam:,occluded,cam,partnercam
0,8\cam1\cam00001_img00524.jpg,1920,1080,"{'bboxes': [[139.0, 60.0, 176.0, 122.0], [172....",17,41,382,329,199,156,"[[379.0, 320.0], [379.0, 321.0], [380.0, 319.0...",1,False,1,karenlazo\cam0\cam00000_img00524.jpg
1,15\cam0\cam00000_img00674.jpg,1920,1080,"{'bboxes': [[40.0, 149.0, 99.0, 185.0], [42.0,...",24,25,332,182,273,128,"[[326.0, 183.0], [326.0, 184.0], [326.0, 185.0...",0,False,0,ronjobintan\cam1\cam00001_img00673.jpg
2,5\cam0\cam00000_img00602.jpg,1920,1080,"{'bboxes': [[44.0, 162.0, 69.0, 202.0], [118.0...",7,30,454,226,416,239,"[[446.0, 238.0], [446.0, 239.0], [446.0, 240.0...",0,False,0,jhaezminnegayo\cam1\cam00001_img00601.jpg
3,11\cam0\cam00000_img00648.jpg,1920,1080,"{'bboxes': [[109.0, 112.0, 140.0, 185.0], [171...",23,39,329,99,263,193,"[[324.0, 76.0], [324.0, 77.0], [324.0, 78.0], ...",0,False,0,paulayap\cam1\cam00001_img00647.jpg
4,8\cam1\cam00001_img00534.jpg,1920,1080,"{'bboxes': [[139.0, 60.0, 176.0, 122.0], [172....",17,41,382,329,198,156,"[[379.0, 320.0], [379.0, 321.0], [380.0, 319.0...",1,False,1,karenlazo\cam0\cam00000_img00534.jpg


In [8]:
## goo real
limbs = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 4), (0, 17), (5, 6), (11, 12), (17, 12), (17, 11), (6, 8), (8, 10), (5, 7), (7, 9), 
         (12, 14), (14, 16), (11, 13), (13, 15)]

limb_colors = [(255, 0, 255), (255, 165, 165), (165, 165, 255), (0, 0, 255), (85, 85, 85), (0, 255, 165), (255, 255, 0), 
               (0, 255, 255), (0, 255, 0), (255, 0, 0), (255, 165, 0), (0, 165, 255), (255, 0, 165), (165, 255, 0), 
               (165, 255, 165), (165, 165, 0), (165, 85, 85), (85, 0, 85)]

kp_colors = [(255, 0, 255), (165, 165, 255), (255, 165, 165), (0, 255, 165), (0, 0, 255), (255, 0, 165), (255, 255, 0), 
             (165, 255, 0), (255, 165, 0), (0, 255, 255), (0, 165, 255), (165, 85, 85), (165, 255, 165), (85, 0, 85), 
             (165, 165, 0), (0, 255, 0), (255, 0, 0), (255, 255, 255)]




def process_mmdet_results(mmdet_results, cat_id=0):
    """Process mmdet results, and return a list of bboxes.
    :param mmdet_results:
    :param cat_id: category id (default: 0 for human)
    :return: a list of detected bounding boxes
    """
    if isinstance(mmdet_results, tuple):
        det_results = mmdet_results[0]
    else:
        det_results = mmdet_results

    bboxes = det_results[cat_id]

    person_results = []
    for bbox in bboxes:
        person = {}
        person['bbox'] = bbox
        person_results.append(person)

    return person_results



def find_person_pose(image, head_bbox, poses):
    """
    Function using a heuristic to associate a head box to the closest matching pose given a list of poses.
    This is used to automatically detect the pose of a person given its head bounding box location.
    
    Args
        image: input image
        head_bbox: the coordinates of the head bounding box of the person of interest [x_min, y_min, x_max, y_max]
        poses: a list of poses where each pose is an nx3 array where n is the number of keypoints and 3 
        corresponds to (x, y) location and detection score.
        
    Output
        The id of the corresponding pose in the list `poses`, or -1 if the closest detected facial keypoints
        are not inside the head bounding box (e.g. when the person of interest was not detected by the pose model)
    """
    x_min, y_min, x_max, y_max = head_bbox
    head_cx = (x_min + x_max) / 2
    head_cy = (y_min + y_max) / 2
    head = np.array([head_cx, head_cy])
    
    faces = []
    for pose in poses:
        face_kp = pose[[0, 1, 2, 3, 4], :] # ids for the facial keypoints
        face_kp = face_kp[face_kp[:, 2] >= 0.3] # filter out non detected keypoints (to avoid biasing the average)
        if len(face_kp) == 0: # no facial keypoints detected
            face_cx, face_cy = 1e6, 1e6
        else:
            face_cx, face_cy = face_kp[:, :2].mean(axis=0)
        faces.append((face_cx, face_cy))
    faces = np.array(faces)
    dist = np.linalg.norm(faces - head, axis = 1)
    idx = np.argmin(dist)
    
    return idx


def draw_pose(image, pose_results, blank=True, kp_thresh=0.3):
    """
    Utility function used to draw detected poses (keypoints and limbs) using a set of colors.
    Args
        image: input image in (HxWxC) format
        pose_results: a list of dicts of the form {'bbox': x, 'pose': y}. This should be the output of the 
        function `inference_top_down_pose_model`.
        blank: whether to draw on an empty canvas or overlay the pose on the original image.
        kp_thresh: the threshold for the keypoints (and their limbs) to keep in the visualization.
    Output
        (empty) image having the same dimension as the input with overlaid poses.
    """
    H, W, C = image.shape
    
    if blank:
        canvas = np.zeros((H, W, 3), dtype=float)
    else:
        canvas = np.copy(image)
        
    
    for result in pose_results:
        # Get Pose and Bbox
        bbox = result['bbox']
        pose = result['keypoints']
        
        # Compute radius and limb width
        bbox_w = np.abs(bbox[0] - bbox[2])
        radius = int(round(np.sqrt(bbox_w + 1) / 3))
        stickwidth = int(round(np.sqrt(bbox_w + 1) / 3))
                        
        # Compute shoulders midpoint
        keypoint17 = (pose[5] + pose[6]) / 2
        pose = np.vstack([pose, keypoint17])
        
        # Find Key Points to Skip (either because score is lower than thresh or because they're not connected to any detected limbs)
        kp_to_draw = np.array([False] * len(pose)) # 18
        for idx, limb in enumerate(limbs):
            p1, p2 = limb
            if (pose[p1, 2] >= kp_thresh) and (pose[p2, 2] >= kp_thresh):
                kp_to_draw[p1] = True
                kp_to_draw[p2] = True
        
        # Draw Key Points
        for p in range(len(pose)):
            if ~kp_to_draw[p]: 
                continue
            x = int(pose[p, 0])
            y = int(pose[p, 1])
            canvas = cv2.circle(canvas, (x, y), radius, kp_colors[p], thickness=-1)
        
        # Draw Limbs
        for idx, limb in enumerate(limbs):
            p1, p2 = limb
            if (pose[p1, 2] < kp_thresh) or (pose[p2, 2] < kp_thresh): 
                continue

            x1, y1 = int(pose[p1, 0]), int(pose[p1, 1])
            x2, y2 = int(pose[p2, 0]), int(pose[p2, 1])
            cur_canvas = canvas.copy()
            
            mean_x = (x1 + x2) / 2
            mean_y = (y1 + y2) / 2
            length = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
            angle = int(math.degrees(math.atan2(y1 - y2, x1 - x2)))
            
            polygon = cv2.ellipse2Poly((int(mean_x), int(mean_y)), (int(length / 2), stickwidth), angle, 0, 360, 1)
            cv2.fillConvexPoly(cur_canvas, polygon, limb_colors[idx])
            canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)   
                        
    return canvas
        

def main():
    """
    ## Same as the one for VideoAttentionTarget.
    """

    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    # Set model parameters
    det_config = '/kaggle/input/hrformer/pose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
    pose_config = '/kaggle/input/hrformer/pose/configs/top_down/hrnet/coco/hrnet_w48_coco_256x192.py'
    det_cat_id = 0 # category if for bounding box detection model
    #input_path = '/kaggle/input/goorealdataset/finalrealdatasetImgsV3/finalrealdatasetImgsV3'
    #output_path = '/kaggle/working/goo-real-pose'
    input_path = '/kaggle/input/goorealdataset/finalrealdatasetImgsV3Sparsed/finalrealdatasetImgsV3Sparsed'
    output_path = '/kaggle/working/goo-real-sparse-test-pose'
    det_checkpoint = '/kaggle/input/fasterrcnnhrformerckpt/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
    pose_checkpoint = '/kaggle/input/fasterrcnnhrformerckpt/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
    bbox_thr = 0.3
    kpt_thr = 0.3 
    eye_thr = 0.6
    min_kpt_pose = 6
    # Load Models
    det_model = init_detector(det_config, det_checkpoint, device=device)
    pose_model = init_pose_model(pose_config,pose_checkpoint, device=device)
    identifier = 0 # Unique person ID in the current frame
    
    #filename	width	height	gaze_cx	gaze_cy	ann	seg_mask
    #columns = ['filename', 'width', 'height', 'gaze_cx', 'gaze_cy', 'ann', 'seg_mask']
            
    # Define split string
    split = 'train'
    
    is_train = True if split == 'train' else False
    
    # Output DataFrame (to record eyes and detected pose)
    df_out = pd.DataFrame(columns = ['path', 'left_eye_x', 'left_eye_y', 'right_eye_x', 'right_eye_y', 'pose_min_x', 'pose_min_y', 'pose_max_x', 'pose_max_y'])
    counter = 0
    N = len(df)
    for ix in tqdm.notebook.tqdm(range(N)):
        # Fetch dataframe row
        row = df.iloc[ix]

        # Path magic
        path = PureWindowsPath(row['filename']).as_posix()
        
        subfolder_path_str = os.path.splitext(path)[0].split("/")
        key_filename = subfolder_path_str[0]+'/'+subfolder_path_str[1]
        # Create Logging File
        if not os.path.exists(os.path.join(output_path, key_filename)):
            os.makedirs(os.path.join(output_path, key_filename))
            logger = open(os.path.join(output_path, f'{key_filename}/logs.txt'), 'a+')
            f_out = open(os.path.join(output_path, f'{key_filename}/pose.csv'), 'a+')

        fname, ext = path.split('.')
        img_pose_fname = fname + f'-pose.' + ext
        subject_pose_fname = fname + f'-{identifier}-pose.' + ext
        json_fname = fname + f'-pose.json'
        

        # Create Output folder if doesn't exist
        if not os.path.isdir(os.path.join(output_path, f'{key_filename}')):
            msg = f'Creating folder...\n'
            print(msg)
            logger.write(msg)
            os.makedirs(os.path.join(output_path, f'{key_filename}'), exist_ok=True)
        

        # Read image
        image = Image.open(os.path.join(input_path, path))
        image = np.array(image.convert('RGB').resize((640,480)))
        height, width, channels = image.shape
        # Extract Pose
        try:
            # Find Bounding Boxes
            mmdet_results = inference_detector(det_model, image)
            # Filter out Bounding Boxes that are not Person Class
            person_results = process_mmdet_results(mmdet_results, det_cat_id)
            # Detect Pose
            results, _ = inference_top_down_pose_model(pose_model, image, person_results, bbox_thr=bbox_thr, format='xyxy', dataset='TopDownCocoDataset', return_heatmap=False, outputs=None)
        except Exception as e:
            msg = f"{type(e).__name__} was raised: {e}. ID: {ix} | Path: {key_filename}"
            print(msg)
            logger.write(msg)

            subject_canvas = np.zeros((128, 64, 3), dtype=np.uint8)
            img_canvas = np.zeros(image.shape, dtype=np.uint8)
            image_pose = Image.fromarray(img_canvas, mode='RGB')
            subject_pose = Image.fromarray(subject_canvas, mode='RGB')
            image_pose.save(os.path.join(output_path,img_pose_fname))
            subject_pose.save(os.path.join(output_path,subject_pose_fname))

            # Record Eye Position and Pose Bbox

            df_out.loc[counter] = [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]
            f_out.write(','.join(list(map(str, [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]))))
            f_out.write('\n')
            counter = counter + 1

            # Save json of poses
            output_json = []
            with open(os.path.join(output_path, json_fname), 'w') as fp:
                json.dump(output_json, fp)
            continue

        # If no person is detected, skip this iteration
        if len(results) == 0:
            msg = f'No detected people in the image (ID: {ix} | Path: {path}).\n'
            print(msg)
            logger.write(msg)
            subject_canvas = np.zeros((128, 64, 3), dtype=np.uint8)
            img_canvas = np.zeros(image.shape, dtype=np.uint8)
            image_pose = Image.fromarray(img_canvas, mode='RGB')
            subject_pose = Image.fromarray(subject_canvas, mode='RGB')
            image_pose.save(os.path.join(output_path,img_pose_fname))
            subject_pose.save(os.path.join(output_path,subject_pose_fname))

            # Record Eye Position and Pose Bbox
        
            df_out.loc[counter] = [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]
            f_out.write(','.join(list(map(str, [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]))))
            f_out.write('\n')
            counter = counter + 1

            # Save json of poses
            output_json = []
            with open(os.path.join(output_path, json_fname), 'w') as fp:
                json.dump(output_json, fp)

            continue

        # Retrieve head bounding box coordinates
        head_bbox = list(row.ann.values())[0][-1]
        
        
        
        # Find target person's pose
        poses = [res['keypoints'] for res in results]
        bboxes = [res['bbox'] for res in results]
        idx = find_person_pose(image, head_bbox, poses)
        pose = poses[idx]
        bbox = bboxes[idx]

        # Compute Pose Bounding Box (sometimes keypoint locations are slightly outside the person bbox detected)        
        pose_min_x = max(min(bbox[0], pose[:, 0].min() - 5), 0)
        pose_min_y = max(min(bbox[1], pose[:, 1].min() - 5), 0)
        pose_max_x = min(max(bbox[2], pose[:, 0].max() + 5), width)
        pose_max_y = min(max(bbox[3], pose[:, 1].max() + 5), height)

        # Detect the Eyes
        if pose[1, 2] > eye_thr: # left eye
            left_eye_x = pose[1, 0]
            left_eye_y = pose[1, 1]
        else:
            left_eye_x, left_eye_y = -1, -1

        if pose[2, 2] > eye_thr: # right eye
            right_eye_x = pose[2, 0]
            right_eye_y = pose[2, 1]
        else:
            right_eye_x, right_eye_y = -1, -1

        # Record Eye Position and Pose Bbox
        
        df_out.loc[counter] = [row['filename'], left_eye_x, left_eye_y, right_eye_x, right_eye_y, pose_min_x, 
                                pose_min_y, pose_max_x, pose_max_y] 
        f_out.write(','.join(list(map(str, [row['filename'], left_eye_x, left_eye_y, right_eye_x, 
                                            right_eye_y, pose_min_x, pose_min_y, pose_max_x, pose_max_y]))))  
        f_out.write('\n')
        counter = counter + 1

        # Discard pose not enough keypoints detected
        if (pose[:, 2] >= kpt_thr).sum() < min_kpt_pose:
            msg = f'Subject person pose has less than 6 key points detected (ID: {ix} | Path: {path}).\n'
            print(msg)
            logger.write(msg)
            subject_canvas = np.zeros((128, 64, 3), dtype=np.uint8)
            img_canvas = np.zeros(image.shape, dtype=np.uint8)
            subject_pose = Image.fromarray(subject_canvas, mode='RGB')
            image_pose = Image.fromarray(img_canvas, mode='RGB')
            image_pose.save(os.path.join(output_path,img_pose_fname))
            subject_pose.save(os.path.join(output_path,subject_pose_fname))

            # Save json of poses
            output_json = [{'bbox': res['bbox'].tolist(), 'keypoints': res['keypoints'].tolist()} for res in results]
            with open(os.path.join(output_path, json_fname), 'w') as fp:
                json.dump(output_json, fp)

            continue        

        # Draw subject pose on blank canvas
        image_pose = draw_pose(image, results, blank=True, kp_thresh=kpt_thr)
        subject_pose = draw_pose(image, [results[idx]], blank=True, kp_thresh=kpt_thr)

        # Crop Subject Pose
        subject_pose = subject_pose[int(pose_min_y): int(pose_max_y), int(pose_min_x): int(pose_max_x), :]

        # Convert image and subject pose to PIL images and Save
        image_pose = Image.fromarray(np.uint8(image_pose), mode='RGB')
        subject_pose = Image.fromarray(np.uint8(subject_pose), mode='RGB')
        image_pose.save(os.path.join(output_path,img_pose_fname))
        subject_pose.save(os.path.join(output_path,subject_pose_fname))

        # Save json of poses
        output_json = [{'bbox': res['bbox'].tolist(), 'keypoints': res['keypoints'].tolist()} for res in results]
        with open(os.path.join(output_path, json_fname), 'w') as fp:
            json.dump(output_json, fp)
    
    
    df_out.to_csv(f'{output_path}/master-pose.csv', header=False, index = False)
    print(len(df_out))
    # Close logging file
    logger.close()
    f_out.close()

if __name__ == "__main__":
    main()



load checkpoint from local path: /kaggle/input/fasterrcnnhrformerckpt/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from local path: /kaggle/input/fasterrcnnhrformerckpt/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth


  0%|          | 0/1162 [00:00<?, ?it/s]

No detected people in the image (ID: 28 | Path: 0/cam0/cam00000_img00158.jpg).

No detected people in the image (ID: 228 | Path: 0/cam0/cam00000_img00210.jpg).

Subject person pose has less than 6 key points detected (ID: 272 | Path: 6/cam0/cam00000_img00719.jpg).

No detected people in the image (ID: 462 | Path: 11/cam1/cam00001_img00633.jpg).

Subject person pose has less than 6 key points detected (ID: 466 | Path: 6/cam0/cam00000_img00243.jpg).

Subject person pose has less than 6 key points detected (ID: 506 | Path: 6/cam0/cam00000_img00428.jpg).

Subject person pose has less than 6 key points detected (ID: 564 | Path: 11/cam0/cam00000_img00627.jpg).

Subject person pose has less than 6 key points detected (ID: 743 | Path: 6/cam0/cam00000_img00592.jpg).

Subject person pose has less than 6 key points detected (ID: 745 | Path: 8/cam1/cam00001_img00262.jpg).

Subject person pose has less than 6 key points detected (ID: 908 | Path: 11/cam1/cam00001_img00204.jpg).

1162


In [9]:
'''
## goo synth
limbs = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 4), (0, 17), (5, 6), (11, 12), (17, 12), (17, 11), (6, 8), (8, 10), (5, 7), (7, 9), 
         (12, 14), (14, 16), (11, 13), (13, 15)]

limb_colors = [(255, 0, 255), (255, 165, 165), (165, 165, 255), (0, 0, 255), (85, 85, 85), (0, 255, 165), (255, 255, 0), 
               (0, 255, 255), (0, 255, 0), (255, 0, 0), (255, 165, 0), (0, 165, 255), (255, 0, 165), (165, 255, 0), 
               (165, 255, 165), (165, 165, 0), (165, 85, 85), (85, 0, 85)]

kp_colors = [(255, 0, 255), (165, 165, 255), (255, 165, 165), (0, 255, 165), (0, 0, 255), (255, 0, 165), (255, 255, 0), 
             (165, 255, 0), (255, 165, 0), (0, 255, 255), (0, 165, 255), (165, 85, 85), (165, 255, 165), (85, 0, 85), 
             (165, 165, 0), (0, 255, 0), (255, 0, 0), (255, 255, 255)]




def process_mmdet_results(mmdet_results, cat_id=0):
    """Process mmdet results, and return a list of bboxes.
    :param mmdet_results:
    :param cat_id: category id (default: 0 for human)
    :return: a list of detected bounding boxes
    """
    if isinstance(mmdet_results, tuple):
        det_results = mmdet_results[0]
    else:
        det_results = mmdet_results

    bboxes = det_results[cat_id]

    person_results = []
    for bbox in bboxes:
        person = {}
        person['bbox'] = bbox
        person_results.append(person)

    return person_results



def find_person_pose(image, head_bbox, poses):
    """
    Function using a heuristic to associate a head box to the closest matching pose given a list of poses.
    This is used to automatically detect the pose of a person given its head bounding box location.
    
    Args
        image: input image
        head_bbox: the coordinates of the head bounding box of the person of interest [x_min, y_min, x_max, y_max]
        poses: a list of poses where each pose is an nx3 array where n is the number of keypoints and 3 
        corresponds to (x, y) location and detection score.
        
    Output
        The id of the corresponding pose in the list `poses`, or -1 if the closest detected facial keypoints
        are not inside the head bounding box (e.g. when the person of interest was not detected by the pose model)
    """
    x_min, y_min, x_max, y_max = head_bbox
    head_cx = (x_min + x_max) / 2
    head_cy = (y_min + y_max) / 2
    head = np.array([head_cx, head_cy])
    
    faces = []
    for pose in poses:
        face_kp = pose[[0, 1, 2, 3, 4], :] # ids for the facial keypoints
        face_kp = face_kp[face_kp[:, 2] >= 0.3] # filter out non detected keypoints (to avoid biasing the average)
        if len(face_kp) == 0: # no facial keypoints detected
            face_cx, face_cy = 1e6, 1e6
        else:
            face_cx, face_cy = face_kp[:, :2].mean(axis=0)
        faces.append((face_cx, face_cy))
    faces = np.array(faces)
    dist = np.linalg.norm(faces - head, axis = 1)
    idx = np.argmin(dist)
    
    return idx


def draw_pose(image, pose_results, blank=True, kp_thresh=0.3):
    """
    Utility function used to draw detected poses (keypoints and limbs) using a set of colors.
    Args
        image: input image in (HxWxC) format
        pose_results: a list of dicts of the form {'bbox': x, 'pose': y}. This should be the output of the 
        function `inference_top_down_pose_model`.
        blank: whether to draw on an empty canvas or overlay the pose on the original image.
        kp_thresh: the threshold for the keypoints (and their limbs) to keep in the visualization.
    Output
        (empty) image having the same dimension as the input with overlaid poses.
    """
    H, W, C = image.shape
    
    if blank:
        canvas = np.zeros((H, W, 3), dtype=float)
    else:
        canvas = np.copy(image)
        
    
    for result in pose_results:
        # Get Pose and Bbox
        bbox = result['bbox']
        pose = result['keypoints']
        
        # Compute radius and limb width
        bbox_w = np.abs(bbox[0] - bbox[2])
        radius = int(round(np.sqrt(bbox_w + 1) / 3))
        stickwidth = int(round(np.sqrt(bbox_w + 1) / 3))
                        
        # Compute shoulders midpoint
        keypoint17 = (pose[5] + pose[6]) / 2
        pose = np.vstack([pose, keypoint17])
        
        # Find Key Points to Skip (either because score is lower than thresh or because they're not connected to any detected limbs)
        kp_to_draw = np.array([False] * len(pose)) # 18
        for idx, limb in enumerate(limbs):
            p1, p2 = limb
            if (pose[p1, 2] >= kp_thresh) and (pose[p2, 2] >= kp_thresh):
                kp_to_draw[p1] = True
                kp_to_draw[p2] = True
        
        # Draw Key Points
        for p in range(len(pose)):
            if ~kp_to_draw[p]: 
                continue
            x = int(pose[p, 0])
            y = int(pose[p, 1])
            canvas = cv2.circle(canvas, (x, y), radius, kp_colors[p], thickness=-1)
        
        # Draw Limbs
        for idx, limb in enumerate(limbs):
            p1, p2 = limb
            if (pose[p1, 2] < kp_thresh) or (pose[p2, 2] < kp_thresh): 
                continue

            x1, y1 = int(pose[p1, 0]), int(pose[p1, 1])
            x2, y2 = int(pose[p2, 0]), int(pose[p2, 1])
            cur_canvas = canvas.copy()
            
            mean_x = (x1 + x2) / 2
            mean_y = (y1 + y2) / 2
            length = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
            angle = int(math.degrees(math.atan2(y1 - y2, x1 - x2)))
            
            polygon = cv2.ellipse2Poly((int(mean_x), int(mean_y)), (int(length / 2), stickwidth), angle, 0, 360, 1)
            cv2.fillConvexPoly(cur_canvas, polygon, limb_colors[idx])
            canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)   
                        
    return canvas
        

def main():
    """
    Same as the one for VideoAttentionTarget.
    """

    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    # Set model parameters
    det_config = '/kaggle/input/hrformer/pose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py'
    pose_config = '/kaggle/input/hrformer/pose/configs/top_down/hrnet/coco/hrnet_w48_coco_256x192.py'
    det_cat_id = 0 # category if for bounding box detection model
    input_path = '/kaggle/input/goosynthtestdataset/goo-synth-test-images/images'
    output_path = '/kaggle/working/goo-synth-test-pose'
    det_checkpoint = '/kaggle/input/fasterrcnnhrformerckpt/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
    pose_checkpoint = '/kaggle/input/fasterrcnnhrformerckpt/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth'
    bbox_thr = 0.3
    kpt_thr = 0.3 
    eye_thr = 0.6
    min_kpt_pose = 6
    # Load Models
    det_model = init_detector(det_config, det_checkpoint, device=device)
    pose_model = init_pose_model(pose_config,pose_checkpoint, device=device)
    identifier = 0 # Unique person ID in the current frame
    
    #filename	width	height	gaze_cx	gaze_cy	ann	seg_mask
    #columns = ['filename', 'width', 'height', 'gaze_cx', 'gaze_cy', 'ann', 'seg_mask']
            
    # Define split string
    split = 'train'
    
    is_train = True if split == 'train' else False
    
    # Output DataFrame (to record eyes and detected pose)
    df_out = pd.DataFrame(columns = ['path', 'left_eye_x', 'left_eye_y', 'right_eye_x', 'right_eye_y', 'pose_min_x', 'pose_min_y', 'pose_max_x', 'pose_max_y'])
    counter = 0
    N = len(df)
    for ix in tqdm.notebook.tqdm(range(N)):
        # Fetch dataframe row
        row = df.iloc[ix]

        # Path magic
        path = row['filename']
        
        # Create Logging File
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            logger = open(os.path.join(output_path, f'/logs.txt'), 'a+')
            f_out = open(os.path.join(output_path, f'/pose.csv'), 'a+')

        fname, ext = path.split('.')
        img_pose_fname = fname + f'-pose.' + ext
        subject_pose_fname = fname + f'-{identifier}-pose.' + ext
        json_fname = fname + f'-pose.json'
        

        # Create Output folder if doesn't exist
        if not os.path.isdir(output_path):
            msg = f'Creating folder...\n'
            print(msg)
            logger.write(msg)
            os.makedirs(output_path, exist_ok=True)
        

        # Read image
        image = Image.open(os.path.join(input_path, path))
        image = np.array(image.convert('RGB').resize((224,224)))
        height, width, channels = image.shape
        # Extract Pose
        try:
            # Find Bounding Boxes
            mmdet_results = inference_detector(det_model, image)
            # Filter out Bounding Boxes that are not Person Class
            person_results = process_mmdet_results(mmdet_results, det_cat_id)
            # Detect Pose
            results, _ = inference_top_down_pose_model(pose_model, image, person_results, bbox_thr=bbox_thr, format='xyxy', dataset='TopDownCocoDataset', return_heatmap=False, outputs=None)
        except Exception as e:
            msg = f"{type(e).__name__} was raised: {e}. ID: {ix} | Path: {key_filename}"
            print(msg)
            logger.write(msg)

            subject_canvas = np.zeros((128, 64, 3), dtype=np.uint8)
            img_canvas = np.zeros(image.shape, dtype=np.uint8)
            image_pose = Image.fromarray(img_canvas, mode='RGB')
            subject_pose = Image.fromarray(subject_canvas, mode='RGB')
            image_pose.save(os.path.join(output_path,img_pose_fname))
            subject_pose.save(os.path.join(output_path,subject_pose_fname))

            # Record Eye Position and Pose Bbox

            df_out.loc[counter] = [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]
            f_out.write(','.join(list(map(str, [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]))))
            f_out.write('\n')
            counter = counter + 1

            # Save json of poses
            output_json = []
            with open(os.path.join(output_path, json_fname), 'w') as fp:
                json.dump(output_json, fp)
            continue

        # If no person is detected, skip this iteration
        if len(results) == 0:
            msg = f'No detected people in the image (ID: {ix} | Path: {path}).\n'
            print(msg)
            logger.write(msg)
            subject_canvas = np.zeros((128, 64, 3), dtype=np.uint8)
            img_canvas = np.zeros(image.shape, dtype=np.uint8)
            image_pose = Image.fromarray(img_canvas, mode='RGB')
            subject_pose = Image.fromarray(subject_canvas, mode='RGB')
            image_pose.save(os.path.join(output_path,img_pose_fname))
            subject_pose.save(os.path.join(output_path,subject_pose_fname))

            # Record Eye Position and Pose Bbox
        
            df_out.loc[counter] = [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]
            f_out.write(','.join(list(map(str, [row['filename'], -1, -1, -1, -1, -1, -1, -1, -1]))))
            f_out.write('\n')
            counter = counter + 1

            # Save json of poses
            output_json = []
            with open(os.path.join(output_path, json_fname), 'w') as fp:
                json.dump(output_json, fp)

            continue

        # Retrieve head bounding box coordinates
        head_bbox = list(row.ann.values())[0][-1]
        
        
        
        # Find target person's pose
        poses = [res['keypoints'] for res in results]
        bboxes = [res['bbox'] for res in results]
        idx = find_person_pose(image, head_bbox, poses)
        pose = poses[idx]
        bbox = bboxes[idx]

        # Compute Pose Bounding Box (sometimes keypoint locations are slightly outside the person bbox detected)        
        pose_min_x = max(min(bbox[0], pose[:, 0].min() - 5), 0)
        pose_min_y = max(min(bbox[1], pose[:, 1].min() - 5), 0)
        pose_max_x = min(max(bbox[2], pose[:, 0].max() + 5), width)
        pose_max_y = min(max(bbox[3], pose[:, 1].max() + 5), height)

        # Detect the Eyes
        if pose[1, 2] > eye_thr: # left eye
            left_eye_x = pose[1, 0]
            left_eye_y = pose[1, 1]
        else:
            left_eye_x, left_eye_y = -1, -1

        if pose[2, 2] > eye_thr: # right eye
            right_eye_x = pose[2, 0]
            right_eye_y = pose[2, 1]
        else:
            right_eye_x, right_eye_y = -1, -1

        # Record Eye Position and Pose Bbox
        
        df_out.loc[counter] = [row['filename'], left_eye_x, left_eye_y, right_eye_x, right_eye_y, pose_min_x, 
                                pose_min_y, pose_max_x, pose_max_y] 
        f_out.write(','.join(list(map(str, [row['filename'], left_eye_x, left_eye_y, right_eye_x, 
                                            right_eye_y, pose_min_x, pose_min_y, pose_max_x, pose_max_y]))))  
        f_out.write('\n')
        counter = counter + 1

        # Discard pose not enough keypoints detected
        if (pose[:, 2] >= kpt_thr).sum() < min_kpt_pose:
            msg = f'Subject person pose has less than 6 key points detected (ID: {ix} | Path: {path}).\n'
            print(msg)
            logger.write(msg)
            subject_canvas = np.zeros((128, 64, 3), dtype=np.uint8)
            img_canvas = np.zeros(image.shape, dtype=np.uint8)
            subject_pose = Image.fromarray(subject_canvas, mode='RGB')
            image_pose = Image.fromarray(img_canvas, mode='RGB')
            image_pose.save(os.path.join(output_path,img_pose_fname))
            subject_pose.save(os.path.join(output_path,subject_pose_fname))

            # Save json of poses
            output_json = [{'bbox': res['bbox'].tolist(), 'keypoints': res['keypoints'].tolist()} for res in results]
            with open(os.path.join(output_path, json_fname), 'w') as fp:
                json.dump(output_json, fp)

            continue        

        # Draw subject pose on blank canvas
        image_pose = draw_pose(image, results, blank=True, kp_thresh=kpt_thr)
        subject_pose = draw_pose(image, [results[idx]], blank=True, kp_thresh=kpt_thr)

        # Crop Subject Pose
        subject_pose = subject_pose[int(pose_min_y): int(pose_max_y), int(pose_min_x): int(pose_max_x), :]

        # Convert image and subject pose to PIL images and Save
        image_pose = Image.fromarray(np.uint8(image_pose), mode='RGB')
        subject_pose = Image.fromarray(np.uint8(subject_pose), mode='RGB')
        image_pose.save(os.path.join(output_path,img_pose_fname))
        subject_pose.save(os.path.join(output_path,subject_pose_fname))

        # Save json of poses
        output_json = [{'bbox': res['bbox'].tolist(), 'keypoints': res['keypoints'].tolist()} for res in results]
        with open(os.path.join(output_path, json_fname), 'w') as fp:
            json.dump(output_json, fp)
    
    
    df_out.to_csv(f'{output_path}/master-pose.csv', header=False, index = False)
    print(len(df_out))
    # Close logging file
    logger.close()
    f_out.close()

if __name__ == "__main__":
    main()
'''

'\n## goo synth\nlimbs = [(0, 1), (0, 2), (1, 3), (2, 4), (3, 4), (0, 17), (5, 6), (11, 12), (17, 12), (17, 11), (6, 8), (8, 10), (5, 7), (7, 9), \n         (12, 14), (14, 16), (11, 13), (13, 15)]\n\nlimb_colors = [(255, 0, 255), (255, 165, 165), (165, 165, 255), (0, 0, 255), (85, 85, 85), (0, 255, 165), (255, 255, 0), \n               (0, 255, 255), (0, 255, 0), (255, 0, 0), (255, 165, 0), (0, 165, 255), (255, 0, 165), (165, 255, 0), \n               (165, 255, 165), (165, 165, 0), (165, 85, 85), (85, 0, 85)]\n\nkp_colors = [(255, 0, 255), (165, 165, 255), (255, 165, 165), (0, 255, 165), (0, 0, 255), (255, 0, 165), (255, 255, 0), \n             (165, 255, 0), (255, 165, 0), (0, 255, 255), (0, 165, 255), (165, 85, 85), (165, 255, 165), (85, 0, 85), \n             (165, 165, 0), (0, 255, 0), (255, 0, 0), (255, 255, 255)]\n\n\n\n\ndef process_mmdet_results(mmdet_results, cat_id=0):\n    """Process mmdet results, and return a list of bboxes.\n    :param mmdet_results:\n    :param cat_id: 

In [10]:
!ls /kaggle/working

__notebook_source__.ipynb  goo-real-sparse-test-pose


In [11]:
!zip -r goo-real-sparse-test-pose.zip /kaggle/working/goo-real-sparse-test-pose


  adding: kaggle/working/goo-real-sparse-test-pose/ (stored 0%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/ (stored 0%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/ (stored 0%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00602-pose.json (deflated 53%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00503-pose.json (deflated 54%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00465-pose.json (deflated 56%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00052-0-pose.jpg (deflated 9%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00044-pose.json (deflated 57%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00494-pose.json (deflated 53%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00128-0-pose.jpg (deflated 5%)
  adding: kaggle/working/goo-real-sparse-test-pose/5/cam0/cam00000_img00115-0-pose.jpg (defl

In [12]:
 from IPython.display import FileLink
FileLink(r'goo-real-sparse-test-pose.zip')

/kaggle/working/goo-real-sparse-test-pose.zip